In [16]:
import torch
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from sklearn.model_selection import train_test_split
from visdom import Visdom
import torch.nn.functional as F
from torch.nn.init import xavier_uniform_
%run Utility_Functions.ipynb
%run parameters.py

In [3]:
# Data import 
x = np.load('X.npy')
y = np.load('Y.npy')

# Select only 50% of the data
x = x[0:5000, :,:]
y = y[0:5000, :]

print(x.shape, y.shape, sum(y)/len(y))
print(np.min(x), np.max(x), np.mean(x))

# Visualization
viz = Visdom()
opts = dict(legend=['Training','Validation', 'Testing'], xlabel = 'Iteration', ylabel = 'Accuracy', title = 'EEG Classification')
# Device configuration 
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch.cuda.empty_cache()
device # verify which device is being used

(5000, 20, 256) (5000, 2) [0.5074 0.4926]
-116.76673911722449 120.9554109493638 4.3431924723336126e-18


device(type='cuda', index=0)

In [5]:
# Prepare the data for convolution neural network 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.40)
x_test, x_valid, y_test, y_valid = train_test_split(x_test, y_test, test_size = 0.50)


print(sum(y_train)/len(y_train), sum(y_test)/len(y_test))

class custom_dataset(Dataset):
    def __init__(self, x, y):
        super(custom_dataset, self).__init__()
        self.x = torch.from_numpy(x).type(torch.float32)
        self.y = torch.from_numpy(y).type(torch.float32)
    
    def __getitem__(self, ids):
        return self.x[ids], self.y[ids] # 2 index of total force ir index
    
    def __len__(self):
        return self.x.shape[0]

# Create datasets 
train_data = custom_dataset(x_train, y_train)
valid_data = custom_dataset(x_valid, y_valid)
test_data = custom_dataset(x_test, y_test)
    
# Hyper-parameters 
OUTPUT = 2
NUM_EPOCHS = 500
BATCH_SIZE = 32
LEARNING_RATE = 0.0001
# WEIGHT_DECAY = 0.0001
    
# Load datasets 
train_data_iterator = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
valid_data_iterator = DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=True)
test_data_iterator = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)

[0.50666667 0.49333333] [0.513 0.487]


In [18]:
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        xavier_uniform_(m.weight.data)

def accuracy(data_iterator):
    with torch.no_grad():
        total = 0
        length = 0
        for x, y in data_iterator:
            out_put = model(x.to(device))
            out_put = out_put.cpu().detach()
            total += (out_put.argmax(dim=1)==y.argmax(dim=1)).float().sum()
            length += len(y)
        accuracy = total/length
        return accuracy

class ConvNet(nn.Module):
    def __init__(self, OUTPUT):
        super(ConvNet, self).__init__()
        self.net_1 = nn.Sequential(
            nn.Conv2d(1, 40, kernel_size=(1, 15), stride=1),
            nn.Conv2d(40, 40, kernel_size=(10, 1), stride=1, bias=False),
            nn.ELU(),
            nn.BatchNorm2d(40, momentum=0.1, affine=True)
            )
        
        self.net_2 = nn.Sequential(
            nn.AvgPool2d(kernel_size=(1, 75), stride=(1, 15)),
            nn.Conv2d(40, OUTPUT, kernel_size=(11, 12), stride=1),
            nn.Dropout(0.25)
            )
        
    def forward(self, x):
        x = x.view(-1, 1, n_electrodes, epoch_length*s_freq)
        out = self.net_1(x)  
        out = out*out
        out = self.net_2(out)
        out = torch.log(torch.clamp(out, min=1e-6))
        out = torch.squeeze(out)
        out = torch.sigmoid(out)
    
        return out
    
# An instance of model
model = ConvNet(OUTPUT).to(device)
# model.apply(weights_init)

viz.close(win='Accuracy')
  
# Loss and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

i = 0

for epoch in range(NUM_EPOCHS):
    for x_batch, y_batch in train_data_iterator:
        # Send the images and labels to gpu 
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        # Forward pass
        out_put = model(x_batch) 
        loss = criterion(out_put, y_batch)

        # Backward and optimize
        optimizer.zero_grad() # For batch gradient optimisation
        loss.backward()
        optimizer.step()
        
# Calculate the accuracy
    train_accuracy = accuracy(train_data_iterator)
    test_accuracy = accuracy(test_data_iterator)

    i = i + 1 
    viz.line(X = np.column_stack([i]*2), Y = np.column_stack((train_accuracy, test_accuracy)), opts = opts, win='Accuracy', update='append')

KeyboardInterrupt: 